In [390]:
%load_ext autoreload
%autoreload 2

import __confing
__confing.change_to_root_folder()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current directory: c:\Users\Leo\OneDrive\Escritorio\Final Proyect


In [391]:
import pandas as pd
import numpy as np

from src import value_filler as vf
from src import feature_selection as fs
from src import data_procces as dp

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from src import evaluation

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error,roc_auc_score,confusion_matrix
from sklearn.model_selection import KFold

In [392]:
def feature_importance(model, df:pd.DataFrame, target:str, n:int):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    feature_names_original = list(df.columns)
    feature_names_original.remove(target)
    
    max_importance = list(indices[:n])
    
    nueva_lista = [feature_names_original[i] for i in max_importance]
    
    return nueva_lista

In [393]:
def get_performance(model, X_train, X_test, y_train, y_test):
    y_pred = model.predict(X_test)
    roc_auc_train = roc_auc_score(y_train, model.predict(X_train))
    # Validation ROC AUC Score
    acc = accuracy_score(y_test, y_pred)
    roc_auc_val = roc_auc_score(y_test, y_pred)
    print( f"Model Accuracy: {acc}",
            f"Train roc_auc: {roc_auc_train}", 
            f"Test roc_auc: {roc_auc_val}",
            sep='\n', end='\n\n')
    print(confusion_matrix(y_test, y_pred),
    classification_report(y_test, y_pred),
    sep='\n'
)

In [404]:
n='a'
df=pd.read_csv(r'data\custom\all_waves.csv')
target=f'p{n}hosp1y'
drop=[f'p{n}hspnit1y','paoophos1y','paoophosf1y'] # si la wave3 falla quitar estas columnas , 'r3bpsft', 'r3bpref'

In [405]:
# Drop
df = df.drop(drop, axis=1)

df = df.dropna(subset=[target]) 

porcentaje_nulos = df.isnull().sum(axis=1) / len(df.columns)
df = df[(df[target] != 0) | (porcentaje_nulos <= 0.4)]

In [406]:
# fill
y = df[target]
X = df.drop(target,axis=1)
X = fs.fast_fill_2(X)

In [407]:
# feature selection
feature_selected=['padoctor1y','paurina2y','pacholst','pabreast','paprost','papapsm','pamammog','pacage','padrinkcr','padrinkbd','padrinkb','paheight','paweight','pabmi','papaina','papainlv','papainfr','pafatigue','pawheeze','pabreath_m','paswell','pahearaid','pahearing','pasight','paglasses','palunglmt_m','pahrtatlmt','pastroklmt','paarthlmt','palowermoba','pauppermoba','pagrossaa','palgmusaa','pamobilaa','paclims','pasit','paarms','palift','pastoop','pachair','pameals','pashop','pamoney','pameds','pabedhlp','pabed','patoilt','paeat','pabath','pawalkr','padress','pahipcomp','pafall','pafallinj','pamhip','pahipe_m','parxarthr','parxstrok','parxhrtat','pacncrothr','pacncrmeds','pacncrradn','pacncrsurg','pacncrchem','parxdiab','parxdiabo','parxhibp','parechrtatt','parifaany','parfaany','parfcaren','pararcare','padresshlp', 'pawalkhlp', 'pabathehlp', 'paeathlp', 'pabedhlp', 'patoilethlp', 'pamealhlp', 'pashophlp', 'pamedhlp', 'pamoneyhlp','pahibpe','pagender','padiabe','pacancre','palunglmt_m','parxlung_m','pahrtatte','pastroke','paarthre','parifaany','parafaany','pahigov','pasmokev','pasmoken','paoangry','paosleep','paodngr','paodngr','paopace','paoplot','paoalchl','pawthh','paagey','pamomage','padadage','paprmem','parjudg','parorgnz']
X=X[feature_selected]
print(len(feature_selected))

110


In [408]:
# split
X_train, X_test, y_train, y_test=dp.split_data(X, y, test_size=0.2, random_state=40)
X_test_o, y_test_o = X_test, y_test

In [409]:
# resample 
X_train, y_train = dp.apply_resample(X_train, y_train,v=1.20)
X_test, y_test = dp.apply_resample(X_test, y_test,v=1.20)

In [410]:
# smote
X_train, y_train = dp.apply_smote(X_train, y_train)

In [411]:
# scaler
X_train, X_test = dp.apply_standard_scaler(X_train, X_test)

In [412]:
# raise ValueError('comentar el error')
import xgboost as xgb

from sklearn.model_selection import KFold

def train_xgboost_with_kfold(features, labels, model):
    k = 10
    kf = KFold(n_splits=k)
    
    for train_index, test_index in kf.split(features):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        
        # Entrenar el modelo XGBoost
        model.fit(X_train, y_train)
    
    return model

model_k = xgb.XGBClassifier()
model_k= train_xgboost_with_kfold(X_train, y_train,model_k)

y_pred_proba = model_k.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(f'AUC={auc}')
print(y_test.value_counts())

print('')
print('######################################################')
print('')

y_pred_proba = model_k.predict_proba(X_test_o)[:, 1]
auc = roc_auc_score(y_test_o, y_pred_proba)
print(f'AUC={auc}')
print(y_test_o.value_counts())

AUC=0.8121562797082233
pahosp1y
0.0    3502
1.0    2919
Name: count, dtype: int64

######################################################

AUC=0.502674345121568
pahosp1y
0.0    21954
1.0     2919
Name: count, dtype: int64


In [413]:
raise ValueError('comentar el error')

import optuna
from sklearn.model_selection import KFold

import xgboost as xgb


def objective(trial):
    # Definir los hiperparámetros a optimizar
    params = {
        'booster': 'gbtree',
        'eta': trial.suggest_float('eta', 0.01, 0.1),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 1.0),
        
        'max_depth': trial.suggest_int('max_depth', 12, 35),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.01),
        'n_estimators': trial.suggest_int('n_estimators', 600, 1000),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.05, 1.0),
    }
    
    k = 10
    kf = KFold(n_splits=k)
    
    for train_index, test_index in kf.split(features):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        
        model = xgb.XGBClassifier(**params)
        model.fit(X_train, y_train)
    
    y_pred_proba = model.predict_proba(X_test_o)[:, 1]
    auc = roc_auc_score(y_test_o, y_pred_proba)
    return auc

features = X_train
labels = y_train

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

ValueError: comentar el error

In [ ]:
print(study.best_params)

In [ ]:
raise ValueError('comentar el error')

import optuna
from sklearn.model_selection import KFold

import xgboost as xgb


def objective(trial):
    # Definir los hiperparámetros a optimizar
    params = {
        'booster': 'dart',
        # 'eta': trial.suggest_float('eta', 0.01, 0.1),
        # 'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 1.0),
        # 'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 1.0),
        
        'max_depth': trial.suggest_int('max_depth', 12, 30),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.01),
        'n_estimators': trial.suggest_int('n_estimators', 600, 1000),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.05, 1.0),
    }
    
    k = 10
    kf = KFold(n_splits=k)
    
    for train_index, test_index in kf.split(features):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        
        model = xgb.XGBClassifier(**params)
        model.fit(X_train, y_train)
    
    y_pred_proba = model.predict_proba(X_test_o)[:, 1]
    auc = roc_auc_score(y_test_o, y_pred_proba)
    return auc

features = X_train
labels = y_train

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

In [ ]:
# raise ValueError('comentar el error')
feature_selected = feature_importance(model_k, df, target, n=70)
print(feature_selected)

In [ ]:
{'eta': 0.017278777004832203, 'reg_alpha': 0.5675043688405764, 'reg_lambda': 0.4793430182530139, 'max_depth': 17, 'learning_rate': 0.007819974793975336, 'n_estimators': 935, 'subsample': 0.7967742160565803, 'colsample_bytree': 0.302428450643778}
{'eta': 0.02475425607581313, 'reg_alpha': 0.858108471588533, 'reg_lambda': 0.6377242839694902, 'max_depth': 22, 'learning_rate': 0.009207206229839365, 'n_estimators': 917, 'subsample': 0.7791266864861979, 'colsample_bytree': 0.27198345018350145}